In [1]:
import os
import pandas as pd


In [2]:
Data = pd.read_csv('Data/HQ_Data.csv')
Data.head()

,Unnamed: 0,review,hours_played,helpful,funny,recommendation,date,game_name,username
0,1,Never cared much about Warhammer until this ga...,91.5,712.0,116,Recommended,13 September,"Warhammer 40,000: Space Marine 2",userpig\n248 products in account
1,9,"""Blessed is the mind too small for doubt.” War...",68.7,755.0,63,Recommended,6 September,"Warhammer 40,000: Space Marine 2","Meat-King, The Ultimate\n364 products in account"
2,18,After 54 hours of playing since the 4-day Earl...,90.3,141.0,3,Recommended,September 5,"Warhammer 40,000: Space Marine 2",♡ Helena Meow ♡\n905 products in account
3,19,an absolutely bombastic campaign with over the...,120.7,250.0,25,Recommended,September 7,"Warhammer 40,000: Space Marine 2","Angarvin\n1,229 products in account"
4,27,I have almost 70 hours in almost a week. And b...,100.9,182.0,11,Recommended,September 12,"Warhammer 40,000: Space Marine 2",Voenstal


In [3]:
game_name_counts = Data['game_name'].value_counts()
print(game_name_counts)


game_name
Grand Theft Auto V                  1551
Tom Clancy's Rainbow Six® Siege      812
Counter-Strike 2                     780
Dead by Daylight                     645
Call of Duty®: Black Ops III         585
Sea of Thieves: 2024 Edition         555
ELDEN RING                           536
Total War: WARHAMMER III             486
Warframe                             446
Call of Duty®                        445
Apex Legends™                        443
Noita                                403
Wallpaper Engine                     317
Dragon's Dogma 2                     224
NARAKA: BLADEPOINT                   163
The First Descendant                  82
Once Human                            77
Black Myth: Wukong                    69
Party Animals                         58
Lies of P                             55
Warhammer 40,000: Space Marine 2      25
LOCKDOWN Protocol                      1
Name: count, dtype: int64


In [4]:
import re
def clean_text(text):
    # 转小写
    text = text.lower()
    # 去除 HTML 标签
    text = re.sub(r'<[^>]+>', ' ', text)
    # 去除 URL
    text = re.sub(r'http[s]?://\S+', ' ', text)
    # 去除非字母数字字符（保留空格）
    text = re.sub(r'[^a-z0-9\s]', ' ', text)
    # 合并多余空白并去除首尾空白
    text = re.sub(r'\s+', ' ', text).strip()
    return text
game_names_to_save = [
    "Grand Theft Auto V",
    "Tom Clancy's Rainbow Six® Siege",
    "Counter-Strike 2",
    "Dead by Daylight",
    "Call of Duty®: Black Ops III",
    "Sea of Thieves: 2024 Edition",
    "ELDEN RING",
    "Total War: WARHAMMER III",
    "Warframe",
    "Call of Duty®",
    "Apex Legends™",
    "Noita",
    "Wallpaper Engine",
    "Dragon's Dogma 2",
    "NARAKA: BLADEPOINT"
]


for game_name in game_names_to_save:
    # Filter data for this game and clean the review field, handling missing values
    game_data = Data[Data['game_name'] == game_name].copy()
    # Replace missing/NaN reviews with empty string and clean text
    game_data['review'] = game_data['review'].fillna('').astype(str).apply(clean_text)
    file_name = f"Data/{game_name.replace(' ', '_').replace('®', '').replace(':', '').replace('™', '')}.csv"
    game_data.to_csv(file_name, index=False)


In [22]:
import pandas as pd

total_scores={}

for game_name in game_names_to_save:
    # 构造文件名（与 cell 4 中保持一致）
    sanitized = game_name.replace(' ', '_').replace('®', '').replace(':', '').replace('™', '')
    file_path = f"Data/{sanitized}.csv"
    # 读取 CSV
    df = pd.read_csv(file_path)
    # 如果存在 predicted 列，则累计 True 的数量并除以总评论数
    # Calculate the range for 'hours_played' and 'helpful' for each game
    hours_played_min = df['hours_played'].min()
    hours_played_max = df['hours_played'].max()
    helpful_min = df['helpful'].min()
    helpful_max = df['helpful'].max()

    # Define hyperparameters for weighting based on the range of 'hours_played' and 'helpful'
    hours_played_weight = 0.5  # Weight for hours played
    helpful_weight = 0.5       # Weight for helpful votes

    # Normalize the 'hours_played' and 'helpful' values to a 0-1 scale
    df['normalized_hours_played'] = (df['hours_played'] - hours_played_min) / (hours_played_max - hours_played_min)
    df['normalized_helpful'] = (df['helpful'] - helpful_min) / (helpful_max - helpful_min)

    # Calculate the weighted score for each review
    df['weighted_score'] = (df['normalized_hours_played'] * hours_played_weight) + \
                           (df['normalized_helpful'] * helpful_weight)
    df['sentiment_score'] = df['predicted'].map(lambda x: 1 if str(x) == 'True' else -1)
    # Calculate the product of 'weighted_score' and 'sentiment_score' for each review
    df['weighted_sentiment'] = df['weighted_score'] * df['sentiment_score']
    
    # Sum the weighted sentiment scores
    # print(f"{game_name}, weighted_score: {df['weighted_score']}")
    # print(f"{game_name}, sentiment_score: {df['sentiment_score']}")
    true_count = df['predicted'].sum()
    false_count = len(df) - true_count
    print(f"{game_name},true_count: {true_count},false_count: {false_count}")
    total_pred = df['predicted'].map(lambda x: 1 if str(x)== 'True' else -1).sum()
    total_reviews = len(df)
    ratio = total_pred
    print(f"{game_name}: {ratio:.2f}")
    total_weighted_sentiment = df['weighted_sentiment'].sum()/total_reviews
    print(f"{game_name}, total_weighted_sentiment: {total_weighted_sentiment:.5f}\n")
    total_scores[game_name] = float(total_weighted_sentiment)

print(total_scores)
# Normalize the total_scores values to a 0-1 scale


# Sort the normalized scores from high to low
sorted_normalized_scores = sorted(total_scores.items(), key=lambda x: x[1], reverse=True)

# Print the sorted normalized scores
for game, score in sorted_normalized_scores:
    print(f"{game}: {score:.5f}")




Grand Theft Auto V,true_count: 521,false_count: 1030
Grand Theft Auto V: -509.00
Grand Theft Auto V, total_weighted_sentiment: -0.07001

Tom Clancy's Rainbow Six® Siege,true_count: 421,false_count: 391
Tom Clancy's Rainbow Six® Siege: 30.00
Tom Clancy's Rainbow Six® Siege, total_weighted_sentiment: 0.01851

Counter-Strike 2,true_count: 398,false_count: 382
Counter-Strike 2: 16.00
Counter-Strike 2, total_weighted_sentiment: 0.02044

Dead by Daylight,true_count: 265,false_count: 380
Dead by Daylight: -115.00
Dead by Daylight, total_weighted_sentiment: -0.03654

Call of Duty®: Black Ops III,true_count: 345,false_count: 240
Call of Duty®: Black Ops III: 105.00
Call of Duty®: Black Ops III, total_weighted_sentiment: 0.02307

Sea of Thieves: 2024 Edition,true_count: 300,false_count: 255
Sea of Thieves: 2024 Edition: 45.00
Sea of Thieves: 2024 Edition, total_weighted_sentiment: 0.01907

ELDEN RING,true_count: 305,false_count: 231
ELDEN RING: 74.00
ELDEN RING, total_weighted_sentiment: 0.02252